In [1]:
%pip install torch pandas torchvision scikit-learn tqdm kaggle -q

Note: you may need to restart the kernel to use updated packages.


In [2]:
!apt update -qq
!apt install -qq unzip
!unzip -q data.zip -d data

E: Could not open lock file /var/lib/apt/lists/lock - open (13: Permission denied)
E: Unable to lock directory /var/lib/apt/lists/
W: Problem unlinking the file /var/cache/apt/pkgcache.bin - RemoveCaches (13: Permission denied)
W: Problem unlinking the file /var/cache/apt/srcpkgcache.bin - RemoveCaches (13: Permission denied)
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


In [4]:
import os
import shutil

import pandas as pd

# Define paths
data_root = "data"
images_dir = os.path.join(data_root, "images")

# Create images directory if it doesn't exist
os.makedirs(images_dir, exist_ok=True)

# List to store image paths and labels
dataset = []

# Loop through each subfolder
for subfolder in os.listdir(data_root):
    subfolder_path = os.path.join(data_root, subfolder)

    # Ensure it's a directory
    if os.path.isdir(subfolder_path) and subfolder != "images":
        # Loop through images inside the subfolder
        for image in os.listdir(subfolder_path):
            old_image_path = os.path.join(subfolder_path, image)

            # Ensure it's a file (image)
            if os.path.isfile(old_image_path):
                # Define new image path in "data/images" directory
                new_image_path = os.path.join(images_dir, image)

                # If filename already exists, rename it to avoid conflicts
                if os.path.exists(new_image_path):
                    base, ext = os.path.splitext(image)
                    counter = 1
                    while os.path.exists(new_image_path):
                        new_image_path = os.path.join(images_dir, f"{base}_{counter}{ext}")
                        counter += 1

                # Move image
                shutil.move(old_image_path, new_image_path)

                # Append to dataset with updated path and original label
                dataset.append({"image_path": new_image_path, "label": subfolder})

        # Optionally remove empty subfolder after moving images
        os.rmdir(subfolder_path)

df = pd.DataFrame(dataset)
df = df.rename(columns={"image_path": "image_id"})
df["image_id"] = df["image_id"].str.replace("data/images/", "", regex=False)

from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df["label"] = label_encoder.fit_transform(df["label"])

df.to_csv(os.path.join(data_root, "dataset.csv"), index=False)

label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

In [ ]:
# To load the dataset again:
# import pandas as pd
# df = pd.read_csv('/workspace/data/dataset.csv')

In [5]:
df["label"].value_counts()

1    1722
7    1194
6     663
5     652
0     471
3     346
8     316
2     314
4     297
Name: label, dtype: int64

In [9]:
import pandas as pd

# Assuming you already have the DataFrame `df`
# and that the column name is actually 'label'

min_count = df['label'].value_counts().min()

# Group by label and take a random sample of size min_count from each class
df_balanced = df.groupby('label').sample(n=min_count, random_state=42)

# Shuffle the resulting balanced dataset
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# Check new class distribution
print(df_balanced['label'].value_counts())


1    297
4    297
5    297
6    297
8    297
2    297
3    297
7    297
0    297
Name: label, dtype: int64


In [10]:
import os

import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from dataset import Dataset


In [19]:
train_df, temp_df = train_test_split(df_balanced, test_size=0.2, random_state=42, stratify=df_balanced["label"])
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df["label"])

# Change the path to the directory where the images are stored
path = "data/images"
train_dataset = Dataset(train_df, path)
test_dataset = Dataset(test_df, path)
val_dataset = Dataset(val_df, path)

In [24]:
import itertools

from model import MaiaNet
from train import Trainer

batch_size = 16
lr = 2e-4
num_epochs = 35
num_classes = 9


def run_experiment(batch_size, lr):
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    model = MaiaNet(num_classes)
    trainer = Trainer(model, train_loader, val_loader, test_loader, lr, num_epochs, batch_size=batch_size)

    trainer.train()
    trainer.test()
    torch.save(trainer.model.state_dict(), 'maianet.pth')

In [ ]:
# for batch_size, lr in itertools.product(batch_sizes, lrs):
#     print(f"\nRunning experiment with batch_size={batch_size}, lr={lr}")
run_experiment(batch_size, lr)

Epoch 1/35: 100%|██████████| 134/134 [01:49<00:00,  1.22it/s, loss=2.2222]



Train Metrics:
--------------------------------------------------
Epoch: 0
Train Loss: 2.1983
Test Loss: 2.1830
Accuracy: 0.1573
Precision: 0.0366
Recall: 0.1573
F1: 0.0569
--------------------------------------------------


Epoch 3/35: 100%|██████████| 134/134 [01:51<00:00,  1.21it/s, loss=2.1533]



Train Metrics:
--------------------------------------------------
Epoch: 2
Train Loss: 2.1453
Test Loss: 2.1310
Accuracy: 0.1985
Precision: 0.0586
Recall: 0.1985
F1: 0.0875
--------------------------------------------------


Epoch 4/35: 100%|██████████| 134/134 [01:50<00:00,  1.22it/s, loss=2.1335]


...

Epoch 13/35: 100%|██████████| 134/134 [01:50<00:00,  1.21it/s, loss=2.0358]



Train Metrics:
--------------------------------------------------
Epoch: 12
Train Loss: 1.9269
Test Loss: 1.9183
Accuracy: 0.4082
Precision: 0.3517
Recall: 0.4082
F1: 0.3158
--------------------------------------------------


Epoch 14/35: 100%|██████████| 134/134 [01:50<00:00,  1.22it/s, loss=1.8496]



Train Metrics:
--------------------------------------------------
Epoch: 13
Train Loss: 1.9073
Test Loss: 1.8935
Accuracy: 0.4345
Precision: 0.2930
Recall: 0.4345
F1: 0.3269
--------------------------------------------------


Epoch 15/35:  48%|████▊     | 64/134 [00:52<00:57,  1.22it/s, loss=1.8434]